## Notebook for collating metadata and analysed data from all experiments and plotting the results

In [1]:
%run ./rob_setup_notebook.ipynb

import utils.interareal_plotting as ip

try:
    with open(master_path, 'rb') as f:
        master_obj = pickle.load(f)
except: 
    raise Exception('ERROR: Could not load master pickle')


Suite2p path: ['/home/rlees/Documents/Code/suite2p/suite2p']
QNAP_path: /home/rlees/mnt/qnap 
QNAP_data_path /home/rlees/mnt/qnap/Data 
Pickle folder: /home/rlees/mnt/qnap/pkl_files 
Master pickle: /home/rlees/mnt/qnap/master_pkl/master_obj.pkl 
Figure save path: /home/rlees/mnt/qnap/Analysis/Figures 
STA movie save path: /home/rlees/mnt/qnap/Analysis/STA_movies 
S2 borders path: /home/rlees/mnt/qnap/Analysis/S2_borders


### See the variables that are attributed to the master object

In [8]:
print('Attributes of master object:', end='\n\n')
for key, value in vars(master_obj).items():
    print(key, len(value))

Attributes of master object:

pkl_folder 30
pkl_name 15
pkl_path 15
sheet_name 48
stim_type 48
tiff_path 48
fps 48
n_units 48
cell_id 48
cell_med 48
s1_cell 48
s2_cell 48
num_s1_cells 48
num_s2_cells 48
stim_dur 48
stim_freq 48
pre_frames 48
post_frames 48
duration_frames 48
time 48
all_trials 48
stas 48
trial_sig_dff 48
trial_sig_dfsf 48
sta_sig 48
sta_sig_nomulti 48
prob_response 48
sta_amplitudes 48
all_amplitudes 48
targeted_cells 48
target_coords 48
n_targets 48
n_targeted_cells 48
trial_target_dff 48
trial_euclid_dist 48
sta_euclid_dist 48
new_pkls 15


### Add new pickle files to the master object

In [ ]:
master_obj.addPickles()

with open(master_path, 'wb') as f:
    pickle.dump(master_obj, f)

### Create master object and collate all analyses results from individual experiments

In [2]:
master_obj = ip.interarealPlotting(pkl_folder)

with open(master_path, 'wb') as f:
    pickle.dump(master_obj, f)

### Convert dictionary of metadata to csv file

In [3]:
keys = []
m_dict = vars(master_obj)

for key, value in m_dict.items():
    if len(value) == 48:
        keys.append(key)

m_dict = { key : m_dict[key] for key in keys }
m_df = pd.DataFrame.from_dict(m_dict)

In [4]:
columns = ['sheet_name',
           'tiff_path',
           'stim_type',
           'n_trials',
           'n_groups',
           'single_stim_dur',
           'n_shots',
           'n_reps',
           'stim_dur',
           'stim_freq',
           'n_targets',
           'n_targeted_cells',
           'spiral_size',
           'n_units',
           'num_s1_cells',
           'num_s2_cells',
           'test_frames',
           'fps',
           'frame_x',
           'frame_y',
           'pix_sz_x',
           'pix_sz_y'
          ]

save_path = os.path.join(fig_save_path, 'BAFOV_OBFOV_exp_metadata.csv')

m_df.to_csv(path_or_buf=save_path, columns=columns, header=True)

### Plot summary statistics and other analytics

In [6]:
with open(master_path, 'rb') as f:
    master_obj = pickle.load(f)

# Plot probability of response on whisker trials for 'ps' and 'pr' target responders
# master_obj.plotWhiskerBias()
    
# Plot mean STA dFF amplitude across all cells in group
# master_obj.boxplotSummaryStat('target_sta_amp')
# master_obj.boxplotSummaryStat('target_responders_sta')
# master_obj.boxplotSummaryStat('n_targeted_cells')
# plt.savefig(os.path.join(fig_save_path, 'target_sta_amp.svg'))
# master_obj.boxplotSummaryStat('non_target_sta_amp')
# master_obj.boxplotSummaryStat('s2_sta_amp')
# master_obj.boxplotSummaryStat('positive_s2_responders_sta')
# master_obj.boxplotSummaryStat('stim_freq')
# plt.savefig(os.path.join(fig_save_path, 'n_targeted_cells.png'))

# Plot distribution of STA amplitudes across all cells in group
# master_obj.distributionSummaryStat('s2_sta')

# Plot mean STA dFF trace across all cells in group
# master_obj.lineplotSTA('target_sta')
# plt.savefig(os.path.join(fig_save_path, 'target_sta.png'))
# master_obj.lineplotSTA('non_target_sta')
# master_obj.lineplotSTA('s2_sta')

# Plot probability of response for each stim separately for each animal
# master_obj.boxplotProbResponse(to_mask='target_cells')
# plt.savefig(os.path.join(fig_save_path, 'prob_response_boxplot.svg'))

# Plot probability of response for each cell across two different stim types
# master_obj.scatterProbResponse(to_mask='target_cells')
# plt.savefig(os.path.join(fig_save_path, 'prob_response_scatter.svg'))

# Plot reliable responders over 100 trials (STA)
# master_obj.scatterResponseSTA(['target_responders_sta', 'positive_s2_responders_sta'], proportion=False)
# master_obj.scatterResponseSTA(['positive_s1_responders_sta', 'positive_s2_responders_sta'], proportion=True)

# Plot all responders per trial (> 2 S.D. of baseline)
# master_obj.scatterResponseTrial(['target_responders_trial_sum', 'positive_s2_responders_trial'], proportion=True)
# master_obj.scatterResponseTrial(['target_responders_trial_sum', 'positive_s2_responders_trial'])
# master_obj.scatterResponseTrial(['positive_s1_responders_trial', 'positive_s2_responders_trial'], proportion=True)
# master_obj.scatterResponseTrial(['positive_s1_responders_trial', 'positive_s2_responders_trial'])
# master_obj.scatterResponseTrial(['target_sum_dff', 'positive_s2_responders_trial'])
# master_obj.scatterResponseTrial(['num_whisker_targets', 'positive_s2_responders_trial'])
# master_obj.scatterResponseTrial(['euclid_dist', 'positive_s2_responders_trial'])

# Plot whisker bias of targets
master_obj.boxplotWhiskerBias()
# plt.savefig(os.path.join(fig_save_path, 'prob_response_whisker_targets.png'))


Number of cells responding out of total targeted:


AttributeError: 'interarealPlotting' object has no attribute 'df'

In [7]:
df = master_obj.df
print(df.columns)

[autoreload of utils.interareal_plotting failed: Traceback (most recent call last):
  File "/home/rlees/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/rlees/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/home/rlees/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/home/rlees/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/home/rlees/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 312, in update_instances
    update_instances(old, new, obj.__dict__, visited)
  File "/home/rlees/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_insta

AttributeError: 'interarealPlotting' object has no attribute 'df'

In [12]:
master_obj.df
# master_obj.df[['tiff_path', 'stim_type', 'n_targeted_cells', 'target_responders_sta']]
# master_obj.df.describe().transpose()

,sheet_name,tiff_path,stim_type,fps,n_units,s1_cells,s2_sta,s2_sta_amp,s2_sta_std,s1_sta,...,non_target_sta_amp,non_target_sta_std,target_responders_trial,target_responders_trial_sum,target_responders,target_responders_sta,target_responders_sta_nomulti,target_sum_dff,num_whisker_targets,euclid_dist
0,2019-11-14_J061,2019-11-14_J061_t-007,pr,15.023208,1972,"[False, False, True, False, True, False, False...","[-0.003658379, 0.00063539774, -0.0054637855, -...",-0.000210,"[0.088237114, 0.08113342, 0.08867555, 0.101278...","[-0.00060495717, -0.011596499, 0.003943409, 0....",...,0.000697,"[0.15761642, 0.11305486, 0.11012849, 0.1370268...","[[False, False, False, False, False, False, Fa...","[26, 23, 19, 15, 15, 14, 18, 18, 21, 14, 12, 1...","[False, False, False, False, False, False, Fal...",40.0,48.0,"[90.72529602050781, 53.25263595581055, 52.7428...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[174.18425363957655, 199.31850455261008, 189.5..."
1,2019-11-14_J061,2019-11-14_J061_t-006,ps,15.220141,1972,"[False, False, True, False, True, False, False...","[-0.0005377551, 0.0025310002, -0.002039906, -0...",0.011227,"[0.09598932, 0.07189933, 0.08526749, 0.0837057...","[0.0035585065, 0.003017501, 0.000586165, 0.002...",...,-0.000995,"[0.12791754, 0.107056834, 0.10646157, 0.110333...","[[False, False, False, False, False, False, Fa...","[6, 6, 6, 3, 6, 6, 5, 5, 2, 5, 4, 6, 8, 5, 4, ...","[False, False, False, False, False, False, Fal...",13.0,26.0,"[13.46841812133789, 13.96081829071045, 14.9803...","[3, 3, 3, 1, 3, 1, 2, 3, 0, 2, 1, 3, 4, 3, 1, ...","[164.05285818228762, 61.00005692164923, 164.05..."
2,2019-11-14_J061,2019-11-14_J061_t-003,none,15.222459,1972,"[False, False, True, False, True, False, False...","[-0.00021534761, -0.00061756256, 0.003458449, ...",-0.009963,"[0.094368264, 0.07572447, 0.09788293, 0.105667...","[-0.0055899094, 0.0023804319, 0.0037102231, 0....",...,-0.005575,"[0.12147756, 0.12429676, 0.21026395, 0.1104628...","[[False, False, False, False, False, False, Fa...","[0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, ...","[False, False, False, False, False, False, Fal...",0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.84754180...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 43.0261548..."
3,2019-11-14_J061,2019-11-14_J061_t-002,w,15.223074,1972,"[False, False, True, False, True, False, False...","[0.0044548707, 0.0047829766, 0.013713324, 0.00...",0.051086,"[0.09339453, 0.091046415, 0.085032664, 0.08494...","[0.010248078, 0.008871942, 0.0005454888, 0.008...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-11-14_J063,2019-11-14_J063_t-006,pr,15.023208,2065,"[False, True, False, False, True, False, False...","[-0.005750719, -0.0009832699, 0.0022554917, -0...",-0.011460,"[0.09948302, 0.095246695, 0.10179344, 0.091673...","[-0.0003314136, -0.0017226624, 0.001839266, -0...",...,-0.009544,"[0.10961244, 0.105698295, 0.112285525, 0.10737...","[[False, False, False, False, False, False, Fa...","[15, 14, 13, 10, 10, 14, 13, 12, 12, 13, 10, 1...","[False, False, False, False, False, False, Fal...",24.0,29.0,"[49.177608489990234, 45.83293151855469, 37.890...","[2, 2, 2, 0, 1, 2, 1, 2, 1, 2, 3, 2, 2, 1, 3, ...","[133.2401882983426, 131.20355349219622, 123.61..."
5,2019-11-14_J063,2019-11-14_J063_t-007,ps,15.023208,2065,"[False, True, False, False, True, False, False...","[0.001018213, -0.00042414566, -0.0015158445, 0...",0.005366,"[0.10512956, 0.10013447, 0.09981507, 0.0891142...","[0.00078992994, 0.0033402552, -0.0002484374, 0...",...,0.004180,"[0.112078205, 0.10513712, 0.11150941, 0.102434...","[[False, False, False, False, False, False, Fa...","[1, 1, 1, 0, 0, 0, 1, 3, 1, 3, 4, 0, 1, 1, 1, ...","[False, False, False, False, False, False, Fal...",6.0,13.0,"[0.6584746837615967, 1.8174747228622437, 1.226...","[0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 1, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 53.2342830..."
6,2019-11-14_J063,2019-11-14_J063_t-005,none,15.023208,2065,"[False